In [1]:
# Load contractions model
from pycontractions import Contractions

cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

### Load Data

In [2]:
import pandas as pd
import numpy as np

file1 = 'Data/c1.csv'
file2 = 'Data/c2.csv'
file3 = 'Data/c3.csv'
file4 = 'Data/c4.csv'

In [3]:
df1 = pd.read_csv(file1,sep=",")
df2 = pd.read_csv(file2,sep=",")
df3 = pd.read_csv(file3,sep=",")
dfall = pd.read_csv(file4,sep=",")

df1 = df1.loc[df1['rating'] != 'OTHER']
df2 = df2.loc[df2['rating'] != 'OTHER']
df3 = df3.loc[df3['rating'] != 'OTHER']
dfall = dfall.loc[dfall['rating'] != 'OTHER']

### Preprocess

In [4]:
import re

def preprocess_claim(sentence):
    sentence = sentence.lower().strip()
    translator = str.maketrans('’', "'", '')
    sentence = sentence.translate(translator)
    sentence = re.sub("u\.s\.","united states",sentence)
    sentence = list(cont.expand_texts([sentence],precise=True))[0]
    sentence = re.sub("[^a-zA-Z0-9_.’,]|(?<!\d)\.(?!\d)|(?<!\w)-(?!\w)|(?<!\d)\,(?!\d)",' ',sentence)
    sentence = re.sub(",",'',sentence)
    sentence = re.sub("\.",'',sentence)
    sentence = re.sub(" a ",' ',sentence)
    sentence = re.sub('\s+', ' ', sentence).strip()
    sentence = re.sub(" s ",' ',sentence)
    if sentence[0:5] == 'says ':
        sentence = sentence[5:]
    sentence = ' '.join([w for w in sentence.split() if len(w)>1])
    return sentence

def preprocess_df(df):
    for index in df.index:
        df.at[index,'text'] = preprocess_claim(df.at[index,'text'])
        rating = df.at[index,'rating']
        if rating == 'FALSE':
            df.at[index,'rating'] = 0
        if rating == 'TRUE':
            df.at[index,'rating'] = 1
        if rating == 'MIXTURE':
            df.at[index,'rating'] = 2
    return df

In [5]:
df1 = preprocess_df(df1)
df2 = preprocess_df(df2)
df3 = preprocess_df(df3)
dfall = preprocess_df(dfall)

### ITF

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def get_itf_dict(ngram,mindf,dfall,topics):
    vectorizer = CountVectorizer(ngram_range=ngram,min_df=2)
    vectorizer.fit(dfall['text'])
    itfdict = {key:0 for key in vectorizer.get_feature_names()}
    for key in itfdict.keys():
        for topic in topics:
            for claim in topic['text']:
                cl = ' ' + claim + ' '
                if cl.find(' ' + key + ' ') >= 0:
                    itfdict[key] = itfdict[key]+1
                    break                
    for key in itfdict.keys():
        itfdict[key] = len(topics) / itfdict[key]
    return itfdict

### ICF

In [7]:
def get_icf_dict(ngram,mindf,df):
    vectorizer = CountVectorizer(ngram_range=ngram,min_df=mindf)
    vectorizer.fit(df['text'])
    icfdict = {key:0 for key in vectorizer.get_feature_names()}
    classes = df["rating"].unique()

    dfcldict = {key:df.loc[df["rating"] == key] for key in classes}

    for key in icfdict.keys():
        for cl in classes:
            dfcl = dfcldict[cl]
            for claim in dfcl["text"]:
                text = ' ' + claim + ' '
                if text.find(' ' + key + ' ') >= 0:
                    icfdict[key] = icfdict[key] + 1
                    break
    return icfdict 

### ITF * ICF

In [8]:
def get_itf_icf_dict(itfdict,icfdict):
    itficfdict = { key:itfdict[key] * icfdict[key] for key in icfdict.keys() }
    return itficfdict

### ITF * ICF Vectors

In [9]:
def computeITFICFVector(claim,itficfdict):
    worddict = itficfdict.keys()
    itficfVector = [0.0] * len(worddict)
    for i, word in enumerate(worddict):
        w = ' ' + word + ' '
        if w in ' ' + claim + ' ':
            itficfVector[i] = itficfdict[word]
    return itficfVector

def get_itf_icf_matrix(df,itficfdict):
    vectors = [computeITFICFVector(claim,itficfdict) for claim in df['text']]
    vectors = np.array(vectors)
    return vectors

### Classification

In [21]:
def get_classes(df):
    df_top = df

    class1 = df_top.copy()
    class2 = df_top.copy()
    class2 = df_top.loc[(df_top['rating'] == 0) | (df_top['rating'] == 1)]
    class3 = df_top.copy()

    for index in class3.index:
        rating = class3.at[index,'rating']
        if rating == 1:
            class3.at[index,'rating'] = 0
        if rating == 2:
            class3.at[index,'rating'] = 1

    dfs = dict()
    dfs['3 Classes'] = class1
    dfs['2 Classes'] = class2
    dfs['2 Mixed Classes'] = class3
    return dfs

In [35]:
topics = dict()
topics['Topic 1'] = get_classes(df1)
topics['Topic 2'] = get_classes(df2)
topics['Topic 3'] = get_classes(df3)
topicall = get_classes(dfall)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [44]:
def classify(topics,topicall,topic,key):
    ngram = (1,3)
    mindf = 2
    dfs = [topics[t][key] for t in topics.keys()]
    dfall = topicall[key]
    
    itfdict = get_itf_dict(ngram,mindf,dfall,dfs)
    icfdict = get_icf_dict(ngram,mindf,topics[topic][key])
    itficfdict = get_itf_icf_dict(itfdict,icfdict)
    
    X = get_itf_icf_matrix(topics[topic][key], itficfdict)
    y = topics[topic][key]["rating"]
    y = y.astype('int')
    clf = LinearSVC(random_state=0, tol=1e-5)
    k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
    score = cross_val_score(clf, X, y, cv=k_fold, scoring='accuracy')
    print('%s_%s - Mean accuracy: %f, Deviation: %f' % (topic,key,score.mean(),score.std()))

In [45]:
for topickey in topics.keys():
    for classkey in dfs.keys():
        classify(topics, topicall, topickey, classkey)

Topic 1_3 Classes - Mean accuracy: 0.413062, Deviation: 0.032718


c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Topic 1_2 Classes - Mean accuracy: 0.580841, Deviation: 0.049786
Topic 1_2 Mixed Classes - Mean accuracy: 0.445484, Deviation: 0.043233


c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of

Topic 2_3 Classes - Mean accuracy: 0.449904, Deviation: 0.016160
Topic 2_2 Classes - Mean accuracy: 0.502763, Deviation: 0.019176


...

c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of

Topic 3_2 Classes - Mean accuracy: 0.574858, Deviation: 0.043395


c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of

Topic 3_2 Mixed Classes - Mean accuracy: 0.446817, Deviation: 0.032354


c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\leonardo.moros\env\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
